
This is for the BCI Competetion 2a



In [1]:
import pandas as pd
import numpy as np
from scipy.signal import welch

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report


There are 4 Bands of data (frequencies) we are looking for [Theta, Alpha, Beta, Gamma] 

Theta     4-8 Hz    Drowsiness/cognitive load
Alpha     8-13 Hz   suppressed when you imagine moving 
Beta      13-30 Hz  motor imagenry/brain planning
Gamma     30-45 Hz  cognitive/motor activity signals

It is important to note that the subject is imagining moving, not actually moving 
so we expect Alpha and Beta to change, thus should be our main focus 

We are using Welch method to estimate the power spectral density (PSD), basically how much power the eeg has at each one of those frequencies

Welch Method:
1. Break signal into overlapping windows
2. Compute frequency transform for each window
3. Average them to get a power estimate

In [2]:
def bandpower(data, sf=250):
    # freqs is the array of frq from 0-125 Hz (we are interested in 0-45 Hz)
    # psd is the PSD of the data
    freqs, psd = welch(data, sf, nperseg=250)  # 250 is the sampling frequncy of the data (stated in the pdf)

    # This is to calcualte the AUC of the PSD curve between fmin and fmax
    def bp(fmin, fmax):
        idx = np.logical_and(freqs >= fmin, freqs <= fmax)
        return np.trapz(psd[idx], freqs[idx])   # This is the total power in the band

    # Now it returns the power in each of the bands
    return {
        "theta": bp(4, 8),
        "alpha": bp(8, 13),
        "beta":  bp(13, 30),
        "gamma": bp(30, 45)
    }


In [3]:
# Load Data
df = pd.read_csv("C:/Users/ahmad/OneDrive/General/Obsidian2/Classes/Fall 2025/EE595_AppliedMachineLearning/Project/Applied-Machine-Learning-Project/Dataset/BCI Competition 2a/Trials/A01T_trials.csv")  
# Get unique trials
trial_ids = df["Trial_ID"].unique()

features = []
labels = []

There are 22 channles and we are extracting 4 bands from each so 
22 * 4 = 88 features

In addition, the main benefit of choosing this dataset over our initial dataset, is that we get access to the unsummarized data

My hypothesis for why the model performed poorly to our old data, wasn't just because of the small set, but because the Random Forest wasn't able to draw noticable patterns from it. I had plotted the data to try to extract patterns myself to guide the model more, although it made a difference it wasn't substantial enough to rely on.

With this dataset we have an extra factor of data, time series, which will show the change in a persons thoughts which we can hopefully exploit 

So to make use of time series, we will be extracting this data:
1. Mean       [Average]
2. Variance   [Flucatuation]
3. Skew       [Waveform leaning up or down?]
4. Kurtosis   [If the signal has sharp peaks or flat peaks]
5. Root Mean Square [Strength of signal]


So now we have 5 additional feature categories:
22 * 5 = 110

110 + 88 = 198 total features for the model to use

In [4]:
# Feature Extraction

# Loop over every row with the same trial id
for tid in trial_ids:

    trial_df = df[df["Trial_ID"] == tid]
    label = trial_df["Label"].iloc[0]
    labels.append(label)
    trial_features = {}

    # Exclude Time, Label, Trial_ID, Subject
    channels = [c for c in trial_df.columns if c.startswith("EEG")]

    # Loop over every channel
    for ch in channels:
        x = trial_df[ch].values

        # Calculate bps for each channel
        bp = bandpower(x)
        for key, val in bp.items():
            trial_features[f"{ch}_{key}"] = val
        
        # Time Based features
        trial_features[f"{ch}_mean"] = np.mean(x)
        trial_features[f"{ch}_var"]  = np.var(x)
        trial_features[f"{ch}_skew"] = pd.Series(x).skew()
        trial_features[f"{ch}_kurt"] = pd.Series(x).kurt()

    features.append(trial_features)

C:\Users\ahmad\AppData\Local\Temp\ipykernel_25672\1214607144.py:9: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(psd[idx], freqs[idx])   # This is the total power in the band


In [5]:
# Convert to DataFrame
X = pd.DataFrame(features)
y = np.array(labels)

In [6]:
# TRAIN / TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [7]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Random Forest Model
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# 3 Fold Cross Validation
grid_search = GridSearchCV(
    estimator=rf, 
    param_grid=param_grid, 
    cv=3, 
    n_jobs=-1, 
    verbose=2,
    scoring='accuracy'
)


In [ ]:
print("Starting Grid Search...")
grid_search.fit(X_train, y_train)

print("\nBest Parameters found:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_test)

print("\n--- Test Set Results (Best Model) ---")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Starting Grid Search...
Fitting 3 folds for each of 216 candidates, totalling 648 fits

Best Parameters found: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation Score: 0.5174868990658464

--- Test Set Results (Best Model) ---
Accuracy: 0.39655172413793105
              precision    recall  f1-score   support

           1       0.31      0.27      0.29        15
           2       0.47      0.57      0.52        14
           3       0.17      0.13      0.15        15
           4       0.56      0.64      0.60        14

    accuracy                           0.40        58
   macro avg       0.38      0.40      0.39        58
weighted avg       0.37      0.40      0.38        58



: 

Starting Grid Search...
Fitting 3 folds for each of 216 candidates, totalling 648 fits

Best Parameters found: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation Score: 0.5174868990658464

--- Test Set Results (Best Model) ---
Accuracy: 0.39655172413793105
              precision    recall  f1-score   support

           1       0.31      0.27      0.29        15
           2       0.47      0.57      0.52        14
           3       0.17      0.13      0.15        15
           4       0.56      0.64      0.60        14